recommendation sys


In [9]:
import numpy as np
import pandas as pd

In [10]:
brand_data = pd.read_csv("brand_dataset.csv")
influencer_data = pd.read_csv("influencer_dataset.csv")

In [11]:
brand_data.head()

,id,brand_name,product_category,target_age,target_gender,target_location
0,1,Brand72,Nutrition Products,15-24,Female,Kerala
1,2,Brand88,Smartphones,All,Male,Gujarat
2,3,Brand55,Home Appliances,18-30,Male,Kerala
3,4,Brand84,Gadgets,15-24,Male,Kerala
4,5,Brand43,Home Appliances,40-50,All,Tamil Nadu


In [12]:
brand_data['brand_name'].unique

<bound method Series.unique of 0      Brand72
1      Brand88
2      Brand55
3      Brand84
4      Brand43
        ...   
595     Brand8
596    Brand80
597    Brand29
598     Brand7
599    Brand89
Name: brand_name, Length: 600, dtype: object>

In [13]:
influencer_data.head()

,id,Influencer_name,Handle,Followers,Posts,Likes,Category,Sentiment,audience_location,audience_age,audience_Gender,Platform,Engagement_rate
0,1,Jane Doe,@emma.t,340770,986,31587,Food,Positive,Kerala,15-24,Male,TikTok,9.269302
1,2,Mia Lee,@jane.doe,702603,847,49725,Fashion,Positive,Gujarat,All,All,Twitter,7.077254
2,3,John Smith,@noah.d,113855,241,41587,Food,Positive,Kerala,18-30,Male,Instagram,36.526283
3,4,Emma Taylor,@jane.doe,202147,944,8042,Lifestyle,Negative,Kerala,15-24,All,Facebook,3.978293
4,5,Mason White,@john.smith,495171,508,30531,Lifestyle,Negative,Tamil Nadu,40-50,Female,Instagram,6.165749


In [14]:
brand_data.isnull().sum()

id                  0
brand_name          0
product_category    0
target_age          0
target_gender       0
target_location     0
dtype: int64

In [15]:

influencer_data.isnull().sum()

id                   0
Influencer_name      0
Handle               0
Followers            0
Posts                0
Likes                0
Category             0
Sentiment            0
audience_location    0
audience_age         0
audience_Gender      0
Platform             0
Engagement_rate      0
dtype: int64

merging both tables

In [16]:
merged_data = pd.merge(brand_data, influencer_data, on = 'id', how = 'right')

In [17]:
merged_data.head()

,id,brand_name,product_category,target_age,target_gender,target_location,Influencer_name,Handle,Followers,Posts,Likes,Category,Sentiment,audience_location,audience_age,audience_Gender,Platform,Engagement_rate
0,1,Brand72,Nutrition Products,15-24,Female,Kerala,Jane Doe,@emma.t,340770,986,31587,Food,Positive,Kerala,15-24,Male,TikTok,9.269302
1,2,Brand88,Smartphones,All,Male,Gujarat,Mia Lee,@jane.doe,702603,847,49725,Fashion,Positive,Gujarat,All,All,Twitter,7.077254
2,3,Brand55,Home Appliances,18-30,Male,Kerala,John Smith,@noah.d,113855,241,41587,Food,Positive,Kerala,18-30,Male,Instagram,36.526283
3,4,Brand84,Gadgets,15-24,Male,Kerala,Emma Taylor,@jane.doe,202147,944,8042,Lifestyle,Negative,Kerala,15-24,All,Facebook,3.978293
4,5,Brand43,Home Appliances,40-50,All,Tamil Nadu,Mason White,@john.smith,495171,508,30531,Lifestyle,Negative,Tamil Nadu,40-50,Female,Instagram,6.165749


Based on Search (using Category)

In [18]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rajya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rajya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_stem(text):
    tokens = nltk.word_tokenize(text.lower())
    stem = [stemmer.stem(w) for w in tokens]
    return " ".join(stem)


In [20]:
merged_data['stemmed_tokens'] =merged_data.apply(lambda row: tokenize_stem(row['Category']+ ' ' + row['audience_location']+ ' ' + row['audience_age']+ ' ' + row['Platform']), axis=1)

In [21]:
merged_data.head()

,id,brand_name,product_category,target_age,target_gender,target_location,Influencer_name,Handle,Followers,Posts,Likes,Category,Sentiment,audience_location,audience_age,audience_Gender,Platform,Engagement_rate,stemmed_tokens
0,1,Brand72,Nutrition Products,15-24,Female,Kerala,Jane Doe,@emma.t,340770,986,31587,Food,Positive,Kerala,15-24,Male,TikTok,9.269302,food kerala 15-24 tiktok
1,2,Brand88,Smartphones,All,Male,Gujarat,Mia Lee,@jane.doe,702603,847,49725,Fashion,Positive,Gujarat,All,All,Twitter,7.077254,fashion gujarat all twitter
2,3,Brand55,Home Appliances,18-30,Male,Kerala,John Smith,@noah.d,113855,241,41587,Food,Positive,Kerala,18-30,Male,Instagram,36.526283,food kerala 18-30 instagram
3,4,Brand84,Gadgets,15-24,Male,Kerala,Emma Taylor,@jane.doe,202147,944,8042,Lifestyle,Negative,Kerala,15-24,All,Facebook,3.978293,lifestyl kerala 15-24 facebook
4,5,Brand43,Home Appliances,40-50,All,Tamil Nadu,Mason White,@john.smith,495171,508,30531,Lifestyle,Negative,Tamil Nadu,40-50,Female,Instagram,6.165749,lifestyl tamil nadu 40-50 instagram


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidvectorizer = TfidfVectorizer(tokenizer=tokenize_stem)

def cosine_sim(txt1,txt2):
    tfid_matrix = tfidvectorizer.fit_transform([txt1,txt2])
    return cosine_similarity(tfid_matrix)[0][1]

In [23]:
def search_product(query):
    stemmed_query = tokenize_stem(query)
    #calcualting cosine similarity between query and stemmed tokens columns
    merged_data['similarity'] = merged_data['stemmed_tokens'].apply(lambda x:cosine_sim(stemmed_query,x))
    res = merged_data.sort_values(by=['similarity'],ascending=False).head(10)[['Influencer_name','Category','Handle','Followers','audience_location','audience_age','audience_Gender','Platform','Engagement_rate']]
    return res

In [24]:
# search based on category, audience_location, age, gender

search_product('Gaming Gujrat') 

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Influencer_name,Category,Handle,Followers,audience_location,audience_age,audience_Gender,Platform,Engagement_rate
292,Emma Taylor,Gaming,@olivia.c,448459,Gujarat,30-45,Male,Twitter,2.461986
99,Isabella Moore,Travel,@isabella.m,996589,Gujarat,25-40,Female,Instagram,3.665704
370,Emma Taylor,Travel,@olivia.c,942599,Gujarat,15-24,Female,Instagram,2.150437
437,Emma Taylor,Travel,@mason.w,621645,Gujarat,25-40,Female,Instagram,4.213981
308,Olivia Clark,Health,@john.smith,724486,Gujarat,25-34,Female,Instagram,6.887366
469,Olivia Clark,Gaming,@sophia.w,641938,Gujarat,40-50,All,YouTube,6.172714
331,Isabella Moore,Gaming,@john.smith,969595,Gujarat,22-40,Male,YouTube,0.587462
542,Mason White,Gaming,@mia.lee,707184,Punjab,18-30,Female,Instagram,1.861185
161,Mia Lee,Gaming,@liam.brown,761467,Rajasthan,18-25,All,Instagram,2.105541
597,Emma Taylor,Gaming,@john.smith,822197,Goa,18-25,Male,Instagram,5.059128


Content base

In [25]:
!python -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2024-10-14 01:17:17.070931: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import os
from scipy.sparse import coo_matrix

Tags Creation

In [27]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm")

def clean_and_extract_tags(text):
    doc = nlp(text.lower())
    tags = [token.text for token in doc if token.text.isalnum() and token.text not in STOP_WORDS]
    return ', '.join(tags)

columns_to_extract_tags_from = ['Category', 'audience_location', 'audience_age',	'audience_Gender',	'Platform']

for column in columns_to_extract_tags_from:
    merged_data[column] = merged_data[column].apply(clean_and_extract_tags)

In [28]:
# Concatenate the cleaned tags from all relevant columns
merged_data['Tags'] = merged_data[columns_to_extract_tags_from].apply(lambda row: ', '.join(row), axis=1)

In [29]:
merged_data.head()

,id,brand_name,product_category,target_age,target_gender,target_location,Influencer_name,Handle,Followers,Posts,...,Category,Sentiment,audience_location,audience_age,audience_Gender,Platform,Engagement_rate,stemmed_tokens,similarity,Tags
0,1,Brand72,Nutrition Products,15-24,Female,Kerala,Jane Doe,@emma.t,340770,986,...,food,Positive,kerala,"15, 24",male,tiktok,9.269302,food kerala 15-24 tiktok,0.276835,"food, kerala, 15, 24, male, tiktok"
1,2,Brand88,Smartphones,All,Male,Gujarat,Mia Lee,@jane.doe,702603,847,...,fashion,Positive,gujarat,,,twitter,7.077254,fashion gujarat all twitter,0.628678,"fashion, gujarat, , , twitter"
2,3,Brand55,Home Appliances,18-30,Male,Kerala,John Smith,@noah.d,113855,241,...,food,Positive,kerala,"18, 30",male,instagram,36.526283,food kerala 18-30 instagram,0.543933,"food, kerala, 18, 30, male, instagram"
3,4,Brand84,Gadgets,15-24,Male,Kerala,Emma Taylor,@jane.doe,202147,944,...,lifestyle,Negative,kerala,"15, 24",,facebook,3.978293,lifestyl kerala 15-24 facebook,0.315457,"lifestyle, kerala, 15, 24, , facebook"
4,5,Brand43,Home Appliances,40-50,All,Tamil Nadu,Mason White,@john.smith,495171,508,...,lifestyle,Negative,"tamil, nadu","40, 50",female,instagram,6.165749,lifestyl tamil nadu 40-50 instagram,0.502232,"lifestyle, tamil, nadu, 40, 50, female, instagram"


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix_content = tfidf_vectorizer.fit_transform(merged_data['Tags'])
cosine_similarities_content = cosine_similarity(tfidf_matrix_content,tfidf_matrix_content)

In [40]:
item_category = 'food'
item_index = merged_data[merged_data['Category']==item_category].index[0]

In [41]:
similar_items = list(enumerate(cosine_similarities_content[item_index]))

In [42]:
similar_items = sorted(similar_items, key=lambda x:x[1], reverse=True)
top_similar_items = similar_items[1:10]

recommended_items_indics = [x[0] for x in top_similar_items]

In [44]:
merged_data.iloc[recommended_items_indics][['Influencer_name','Category','Handle','Followers','audience_location','audience_age','audience_Gender','Platform','Engagement_rate']]

,Influencer_name,Category,Handle,Followers,audience_location,audience_age,audience_Gender,Platform,Engagement_rate
214,Olivia Clark,food,@jane.doe,868842,bihar,"15, 24",male,tiktok,4.347856
349,Olivia Clark,music,@emma.t,675046,kerala,"15, 24",,facebook,3.176969
3,Emma Taylor,lifestyle,@jane.doe,202147,kerala,"15, 24",,facebook,3.978293
106,Isabella Moore,food,@john.smith,719823,india,"15, 24",male,youtube,5.772113
565,Isabella Moore,lifestyle,@jane.doe,591266,kerala,"15, 24",female,youtube,7.662710
213,Isabella Moore,gaming,@sophia.w,561010,kerala,"15, 24",female,facebook,2.245949
540,Jane Doe,travel,@jane.doe,871592,haryana,"15, 24",male,tiktok,1.690699
410,Mason White,photography,@john.smith,951799,india,"15, 24",male,tiktok,4.229780
232,Emma Taylor,food,@mia.lee,885122,bihar,"15, 24",,instagram,2.335158


Function to recommend for conten base

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def content_based_recommendations(train_data, item_name, top_n=10):
    # Check if the item name exists in the training data
    if item_name not in merged_data['Category'].values:
        print(f"Item '{item_name}' not found in the training data.")
        return pd.DataFrame()

    # Create a TF-IDF vectorizer for item descriptions
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    # Apply TF-IDF vectorization to item descriptions
    tfidf_matrix_content = tfidf_vectorizer.fit_transform(merged_data['Tags'])

    # Calculate cosine similarity between items based on descriptions
    cosine_similarities_content = cosine_similarity(tfidf_matrix_content, tfidf_matrix_content)

    # Find the index of the item
    item_index = merged_data[merged_data['Category'] == item_name].index[0]

    # Get the cosine similarity scores for the item
    similar_items = list(enumerate(cosine_similarities_content[item_index]))

    # Sort similar items by similarity score in descending order
    similar_items = sorted(similar_items, key=lambda x: x[1], reverse=True)

    # Get the top N most similar items (excluding the item itself)
    top_similar_items = similar_items[1:top_n+1]

    # Get the indices of the top similar items
    recommended_item_indices = [x[0] for x in top_similar_items]

    # Get the details of the top similar items
    recommended_items_details = train_data.iloc[recommended_item_indices][['Influencer_name','Category','Handle','Followers','audience_location','audience_age','audience_Gender','Platform','Engagement_rate']]

    return recommended_items_details

In [48]:
# Example: Get content-based recommendations for a specific item
item_category = 'food'
content_based_rec = content_based_recommendations(merged_data, item_category, top_n=8)

content_based_rec

,Influencer_name,Category,Handle,Followers,audience_location,audience_age,audience_Gender,Platform,Engagement_rate
214,Olivia Clark,food,@jane.doe,868842,bihar,"15, 24",male,tiktok,4.347856
349,Olivia Clark,music,@emma.t,675046,kerala,"15, 24",,facebook,3.176969
3,Emma Taylor,lifestyle,@jane.doe,202147,kerala,"15, 24",,facebook,3.978293
106,Isabella Moore,food,@john.smith,719823,india,"15, 24",male,youtube,5.772113
565,Isabella Moore,lifestyle,@jane.doe,591266,kerala,"15, 24",female,youtube,7.662710
213,Isabella Moore,gaming,@sophia.w,561010,kerala,"15, 24",female,facebook,2.245949
540,Jane Doe,travel,@jane.doe,871592,haryana,"15, 24",male,tiktok,1.690699
410,Mason White,photography,@john.smith,951799,india,"15, 24",male,tiktok,4.229780


Collaborative using item similarity

In [50]:
user_item_matrix = merged_data.pivot_table(index='id', columns='Category', values='Engagement_rate',aggfunc='mean').fillna(0).astype(int)

In [51]:
user_similarity = cosine_similarity(user_item_matrix)

In [52]:
target_user_id = 4
target_user_index = user_item_matrix.index.get_loc(target_user_id)

In [54]:
ser_similarities = user_similarity[target_user_index]

similar_user_indices = user_similarity.argsort()[::-1][1:]

In [59]:
recommend_items = []

for user_index in similar_user_indices:
    rated_by_similar_user = user_item_matrix.iloc[user_index]
    not_rated_by_target_user = (rated_by_similar_user==0) & (user_item_matrix.iloc[target_user_index]==0)
    
    recommend_items.extend(user_item_matrix.columns[not_rated_by_target_user][:10])

recommended_items_details = merged_data[merged_data['id'].isin(recommend_items)][['Category', 'audience_location', 'audience_age',	'audience_Gender',	'Platform']]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
recommended_items_details.head(10)